
    
<span style="margin:auto; display:table;color:black;font-size:18px;font-family:Arial">Fachhochschule Südwestfalen</span>

<span style="margin:auto; display:table;color:black;font-size:18px;font-family:Arial">Semesterabschließende schriftliche Ausarbeitung im Modul Machine Learning (SS2023)</span>

<span style="margin:auto; display:table;color:black;font-size:30px;font-family:Arial"><br><br>Preisvorhersage von Gebrauchtwagen<br>
</span>
    
    
    
<span style="margin:auto; display:table;color:black;font-size:18px;font-family:Arial"><br><br><br><br>Bearbeitet von M.Sc. Onur Yilmaz</span>

<span style="margin:auto; display:table;color:black;font-size:18px;font-family:Arial"><br>10. Juli 2023</span>

In [524]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

import os

In [525]:
import os
import pandas as pd

# Pfad zu den CSV-Dateien
file_path = './datasets'

# Eine leere DataFrame erstellen
df = pd.DataFrame()

# Durchsuche das Verzeichnis und lese die CSV-Dateien ein
for dirname, _, filenames in os.walk(file_path):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        # Überprüfen, ob die Datei leer ist, bevor sie gelesen wird
        if os.stat(path).st_size > 0:
            brand_data = pd.read_csv(path)
            if filename == 'focus.csv':
                brand = "Ford"  # Setze den Brand auf "Ford" für focus.csv
            elif filename == 'cclass.csv':
                brand = "merc"  # Setze den Brand auf "merc" für cclass.csv
            else:
                brand = filename[:-4]  # Extrahiere den Brand-Namen aus dem Dateinamen
            brand_data.insert(0, 'brand', brand)  # Füge die "Brand"-Spalte zum DataFrame hinzu
            df = pd.concat([df, brand_data])

# Den resultierenden DataFrame anzeigen oder speichern
df.to_csv('./clean/merged.csv', index=False)
df

,brand,model,year,price,transmission,mileage,fuelType,engineSize,tax,mpg,tax(£)
0,Ford,Focus,2016,8000,Manual,38852,Petrol,1.0,NaN,NaN,NaN
1,Ford,Focus,2019,13400,Manual,11952,Petrol,1.0,NaN,NaN,NaN
2,Ford,Focus,2019,14600,Manual,22142,Petrol,1.5,NaN,NaN,NaN
3,Ford,Focus,2016,9450,Manual,14549,Diesel,1.6,NaN,NaN,NaN
4,Ford,Focus,2015,9999,Manual,7010,Diesel,1.6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15152,vw,Eos,2012,5990,Manual,74000,Diesel,2.0,125.0,58.9,NaN
15153,vw,Fox,2008,1799,Manual,88102,Petrol,1.2,145.0,46.3,NaN
15154,vw,Fox,2009,1590,Manual,70000,Petrol,1.4,200.0,42.0,NaN
15155,vw,Fox,2006,1250,Manual,82704,Petrol,1.2,150.0,46.3,NaN


### Erste Schritte

In [526]:
df.shape

(108540, 11)

In [527]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108540 entries, 0 to 15156
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   brand         108540 non-null  object 
 1   model         108540 non-null  object 
 2   year          108540 non-null  int64  
 3   price         108540 non-null  int64  
 4   transmission  108540 non-null  object 
 5   mileage       108540 non-null  int64  
 6   fuelType      108540 non-null  object 
 7   engineSize    108540 non-null  float64
 8   tax           94327 non-null   float64
 9   mpg           99187 non-null   float64
 10  tax(£)        4860 non-null    float64
dtypes: float64(4), int64(3), object(4)
memory usage: 9.9+ MB


In [528]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
year,108540.0,2017.098028,2.130057,1970.0,2016.00,2017.0,2019.0,2060.0
price,108540.0,16890.124046,9756.266820,450.0,10229.50,14698.0,20940.0,159999.0
mileage,108540.0,23025.928469,21176.423684,1.0,7491.75,17265.0,32236.0,323000.0
engineSize,108540.0,1.661644,0.557058,0.0,1.20,1.6,2.0,6.6
tax,94327.0,120.256183,63.404805,0.0,125.00,145.0,145.0,580.0
mpg,99187.0,55.166825,16.138522,0.3,47.10,54.3,62.8,470.8
tax(£),4860.0,121.147119,58.003289,0.0,125.00,145.0,145.0,555.0


In [529]:
print('min year:', min(df.year))
print('max year:', max(df.year))

min year: 1970
max year: 2060


In [530]:
df[df.year > 2020]

,brand,model,year,price,transmission,mileage,fuelType,engineSize,tax,mpg,tax(£)
17726,ford,Fiesta,2060,6495,Automatic,54807,Petrol,1.4,205.0,42.8,NaN


In [531]:
df.drop(17726, inplace=True)

In [532]:
df.brand.value_counts()

ford        17964
merc        17018
vw          15157
vauxhall    13632
bmw         10781
audi        10668
toyota       6738
skoda        6267
Ford         5454
hyundi       4860
Name: brand, dtype: int64

In [533]:
# Erstelle das Mapping für die Markennamen
mapping = {
    'ford': 'Ford',
    'vw': 'Volkswagen',
    'vauxhall': 'Vauxhall',
    'merc': 'Mercedes',
    'bmw': 'BMW',
    'audi': 'Audi',
    'toyota': 'Toyota',
    'skoda': 'Skoda',
    'hyundi': 'Hyundai'
}

# Wende das Mapping auf die Spalte 'brand' an
df['brand'] = df['brand'].map(mapping)


In [534]:
df.brand.value_counts()

Ford          17964
Mercedes      17018
Volkswagen    15157
Vauxhall      13632
BMW           10781
Audi          10668
Toyota         6738
Skoda          6267
Hyundai        4860
Name: brand, dtype: int64

In [535]:
df.transmission.unique()

array(['Manual', 'Automatic', 'Semi-Auto', 'Other'], dtype=object)

In [536]:
df.fuelType.unique()


array(['Petrol', 'Diesel', 'Hybrid', 'Electric', 'Other'], dtype=object)

### Datenbereinigung

In [537]:
df.isna().sum()

brand             5454
model                0
year                 0
price                0
transmission         0
mileage              0
fuelType             0
engineSize           0
tax              14213
mpg               9353
tax(£)          103679
dtype: int64

In [538]:
df = df.drop(columns=['tax', 'tax(£)'])

In [539]:
df = df.dropna()

In [540]:
df.isna().sum()

brand           0
model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
engineSize      0
mpg             0
dtype: int64

In [541]:
df

,brand,model,year,price,transmission,mileage,fuelType,engineSize,mpg
0,Ford,Fiesta,2017,12000,Automatic,15944,Petrol,1.0,57.7
1,Ford,Focus,2018,14000,Manual,9083,Petrol,1.0,57.7
2,Ford,Focus,2017,13000,Manual,12456,Petrol,1.0,57.7
3,Ford,Fiesta,2019,17500,Manual,10460,Petrol,1.5,40.3
4,Ford,Fiesta,2019,16500,Automatic,1482,Petrol,1.0,48.7
...,...,...,...,...,...,...,...,...,...
15152,Volkswagen,Eos,2012,5990,Manual,74000,Diesel,2.0,58.9
15153,Volkswagen,Fox,2008,1799,Manual,88102,Petrol,1.2,46.3
15154,Volkswagen,Fox,2009,1590,Manual,70000,Petrol,1.4,42.0
15155,Volkswagen,Fox,2006,1250,Manual,82704,Petrol,1.2,46.3


In [542]:
# mapping = {
#     'brand': 'Marke',
#     'model': 'Modell',
#     'year': 'Jahr',
#     'price': 'Preis',
#     'transmission': 'Getriebe',
#     'mileage': 'Laufleistung',
#     'fuelType': 'Kraftstoffart',
#     'engineSize': 'Motorgröße',
#     'mpg': 'Kraftstoffverbrauch (mpg)'
# }

# # Spaltennamen aktualisieren
# df_german = df.rename(columns=mapping)


### Modell

#### Scaling

In [543]:
sc = StandardScaler()
df['mileage'] = sc.fit_transform(df[['mileage']])
df[['mileage']].head()


,mileage
0,-0.336415
1,-0.660838
2,-0.501345
3,-0.595727
4,-1.020253


In [544]:
df['mpg'] = sc.fit_transform(df[['mpg']])
df[['mpg']].head()

,mpg
0,0.156957
1,0.156957
2,0.156957
3,-0.921212
4,-0.400716


#### Encoding

In [545]:
df = pd.get_dummies(data=df, columns=['brand', 'model', 'transmission', 'fuelType'])

In [546]:
df.shape

(99186, 218)

#### Split Data

In [547]:
X = df.drop(columns='price')
y = df['price']

In [548]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Model-1 Decision Tree Regressor¶


In [549]:
reg_dt = DecisionTreeRegressor(random_state=0)

In [550]:
reg_dt.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [551]:
preds_dt = reg_dt.predict(X_test)

In [552]:
print('R Square Score for Decision Tree:', round(r2_score(y_test, preds_dt)*100, 2), '%')

R Square Score for Decision Tree: 94.32 %


#### Model-2 Random Forest Regressor

In [553]:
reg_rf = RandomForestRegressor(random_state= 0)

In [554]:
reg_rf.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [555]:
preds_rf = reg_rf.predict(X_test)

In [556]:
print('R Square Score for Random Forest: ', round(r2_score(y_test, preds_rf)*100, 2), '%')

R Square Score for Random Forest:  96.11 %
